In [ ]:
import numpy as np
import random
import itertools
import scipy.stats

In [ ]:
# def classify(X,w):
#     P = 1/(1 + np.exp(-np.matmul(X,w)))
#     return np.random.binomial(1,P)

In [ ]:
# def bayes_opt(X, balance):
#     feature_count = {}
#     for r in X:
#         x = str(r[0:len(r)-1])
#         y = r[len(r)-1]
#         if x in feature_count:
#           (p,n,d) = feature_count[x]
#           if y > 0:
#             p += 1
#           else:
#             n += 1
#           if p > n:
#             d = np.random.binomial(1, n/(n+p)) if balance else 1 
#           elif p < n:
#             d = 0
#           else:
#             d = random.randint(0,1)
#           feature_count[x] = (p,n,d)
#         else:
#           if y > 0:
#             feature_count[x] = (1,0,1)
#           else:
#             feature_count[x] = (0,1,1)
#     return feature_count

In [ ]:
# def stat_parity(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return b.sum() / len(b)

# def equal_opp(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return b[Y>0].sum() / len(b[Y>0])

# def equal_unopp(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return b[Y<1].sum() / len(b[Y<1])

# def equal_acc(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return np.square(b - Y).sum() / len(b)

# def pred_value_parity_pos(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return Y[b>0].sum() / len(Y[b>0])

# def pred_value_parity_neg(X, Y, d):
#     b = np.apply_along_axis(lambda x : d[str(x)][2], 1, X)
#     return Y[b<1].sum() / len(Y[b<1])

In [ ]:
# def compute_average(n,w,X,Y):
#     avg_stat_parity = 0
#     avg_equal_opp = 0
#     avg_pred_value_parity_pos = 0
#     for _ in range(n):   
#         D = np.hstack((X,Y.reshape(s,1)))
#         d = bayes_opt(D,False)
#         avg_stat_parity += stat_parity(X,Y,d)
#         avg_equal_opp += equal_opp(X,Y,d)
#         avg_pred_value_parity_pos += pred_value_parity_pos(X,Y,d)
#     avg_stat_parity /= n
#     avg_equal_opp /= n
#     avg_pred_value_parity_pos /= n
#     print("stat_parity", avg_stat_parity)
#     print("equal_opp", avg_equal_opp)
#     print("pred_val_parity_pos", avg_pred_value_parity_pos)

In [ ]:
# n = 100
# w = np.array([0.01,0.01])
# s = 5000
# X = np.random.multinomial(4, [1/2.]*2, size=s)
# Y = classify(X,w)
# compute_average(n,w,X,Y)
# # X = np.random.multinomial(4, [1, 0], size=s)
# # Y = classify(X,w)
# # compute_average(n,w,X,Y)

In [ ]:
# print(X)
# print(Y)
# D = np.hstack((X,Y.reshape(s,1)))
# d = bayes_opt(D,False)
# print(d)

In [ ]:
# n = 100
# w = np.array([0.01,0.01,0.01,0.01,0.01])
# s = 3000
# X = np.random.multinomial(5, [1/5.]*5, size=s)
# Y = classify(X,w)
# compute_average(n,w,X,Y)
# X = np.random.multinomial(5, [0.8, 0.1, 0.1, 0, 0], size = s)
# Y = classify(X,w)
# compute_average(n,w,X,Y)

In [ ]:
# avg = 0
# n = 100
# for _ in range(n):   
#     w = np.array([1,1,1,1,1])
#     X = np.random.multinomial(5, [1/5.]*5, size=1000)
#     Y = classify(X,np.array([0.01,0.01,0.09,0.03,0.01]))
#     D = np.hstack((X,Y.reshape(1000,1)))
#     d = bayes_opt(D,False)
#     avg += stat_parity(X,Y,d)
# avg /= n
# print(avg)

In [ ]:
#All Probability functions give the probability that Y = 1
def Pr_Y_given_X(X,w,b):
    return 1/(1 + np.exp(-(np.matmul(X,w) + b)))

In [ ]:
def partitions(n, b):
    masks = np.identity(b, dtype=int)
    for c in itertools.combinations_with_replacement(masks, n): 
        yield sum(c)
output = np.array(list(partitions(4, 4)))

In [ ]:
def Pr_Y(w,b,probs):
    AllX = np.array(list(partitions(4,4)))
    PrX = scipy.stats.multinomial.pmf(output, n = np.full(AllX.shape[0],4), p = probs)
    return np.dot(Pr_Y_given_X(AllX,w,b), PrX)

In [ ]:
X = np.array([[1,0,3,0],[2,2,0,0]])
w = np.array([0.5, 0, 0.2, 0.1])
b = -0.4
probs = [0.3,0.2,0.4,0.1]
PrY = Pr_Y(w,b,probs)
print(PrY)
probs = [0.6,0.2,0.1,0.1]
PrY = Pr_Y(w,b,probs)
print(PrY)

In [ ]:
def Pr_Yhat_given_Y(w,b,probs,Y):
    AllX = np.array(list(partitions(4,4)))
    PrX = scipy.stats.multinomial.pmf(output, n = np.full(AllX.shape[0],4), p = probs)
    if Y == 1:
        return np.dot(np.square(Pr_Y_given_X(AllX,w,b)),PrX) / np.dot(Pr_Y_given_X(AllX,w,b), PrX)
    else:
        return np.dot(np.multiply(1 - Pr_Y_given_X(AllX,w,b),(Pr_Y_given_X(AllX,w,b))),PrX) / np.dot(Pr_Y_given_X(AllX,w,b), PrX)

In [ ]:
probs = [0.3,0.2,0.4,0.1]
PrYhatGivenY = Pr_Yhat_given_Y(w,b,probs,0)
print(PrYhatGivenY)
PrYhatGivenY = Pr_Yhat_given_Y(w,b,probs,1)
print(PrYhatGivenY)
probs = [0.6,0.2,0.1,0.1]
PrYhatGivenY = Pr_Yhat_given_Y(w,b,probs,0)
print(PrYhatGivenY)
PrYhatGivenY = Pr_Yhat_given_Y(w,b,probs,1)
print(PrYhatGivenY)

In [ ]:
def ExpYminusYsq(w,b,probs):
    return (1 - Pr_Yhat_given_Y(w,b,probs,1))*Pr_Y(w,b,probs) + Pr_Yhat_given_Y(w,b,probs,0)*(1 - Pr_Y(w,b,probs))

In [ ]:
probs = [0.3,0.2,0.4,0.1]
ExpYYsq = ExpYminusYsq(w,b,probs)
print(ExpYYsq)
probs = [0.6,0.2,0.1,0.1]
ExpYYsq = ExpYminusYsq(w,b,probs)
print(ExpYYsq)
probs = [0.1,0.4,0.4,0.1]
ExpYYsq = ExpYminusYsq(w,b,probs)
print(ExpYYsq)